In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# # for dirname, _, filenames in os.walk('/kaggle/input'):
# #     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!apt update
!apt install --yes python-opencv
!apt install --yes libopencv-dev
!/bin/bash -c 'echo "/opt/conda/lib/" > /etc/ld.so.conf.d/opencv.conf'
!ldconfig
!pip install imagesize

In [ ]:
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import ast
import glob
import shutil
import sys
import numpy as np
import imagesize
import cv2
from tqdm.notebook import tqdm

In [ ]:
#Install Darknet
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
#Build Darknet with GPU enable settings
%cd darknet

!cp '../../input/libcuda/libcuda.so' .

!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile

!sed -i 's/LDFLAGS+= -L\/usr\/local\/cuda\/lib64 -lcuda -lcudart -lcublas -lcurand/LDFLAGS+= -L\/usr\/local\/cuda\/lib64 -lcudart -lcublas -lcurand -L\/kaggle\/working\/darknet -lcuda/' Makefile
!make &> compile.log

In [ ]:
#Verify build
!./darknet detector train

In [ ]:
#Define image/label path
ROOT_DIR  = '/kaggle/input'
WORKING_DIR  = '/kaggle/working'
def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/tensorflow-great-barrier-reef/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    row['label_path'] = f'{WORKING_DIR}/darknet/data/obj/video_{row.video_id}_{row.video_frame}.txt'
    return row

In [ ]:
#Load annotations in dataframe
df = pd.read_csv(f'{ROOT_DIR}/tensorflow-great-barrier-reef/train.csv')
df = df.apply(get_path, axis=1)
df['annotations'] = df['annotations'].apply(lambda x: ast.literal_eval(x))
display(df.head(2))

In [ ]:
# Check negative samples also
df['num_bbox'] = df['annotations'].apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100
print('% images without annotations: {}'.format(data[0]))
print('% images with annotations: {} '.format(data[1]))

In [ ]:
#Filter out negative samples
df = df.query("num_bbox>0")

In [ ]:
#Convert given annotations to YOLO format
def coco2yolo(image_height, image_width, bboxes):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normalizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row


df['bboxes'] = df.annotations.apply(get_bbox)
df = df.apply(get_imgsize,axis=1)
display(df.width.unique(), df.height.unique())
display(df.head(2))

In [ ]:
#Maintain the Darknet's YOLO required directory structure
%cd data/
!mkdir obj test

cnt = 0
for row_idx in tqdm(range(df.shape[0])):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width = row.width
    bboxes_coco = np.asarray(row.bboxes).astype(np.float32).copy()
    num_bbox = len(bboxes_coco)
    labels = [0]*num_bbox
  
    f = open(row.label_path, 'w')

    if num_bbox < 1:
        annot = ''
        f.write(annot)
        f.close()
        cnt += 1
        continue
  
    bboxes_yolo  = coco2yolo(image_height, image_width, bboxes_coco)

    for i in range(len(bboxes_yolo)):
        annot = [str(labels[i])] + list(bboxes_yolo[i].astype(str)) + (['\n'] if num_bbox!=(i+1) else [''])
        annot = ' '.join(annot)
        annot = annot.strip(' ')
        f.write(annot)
    f.close()

print('Missing boxes ', cnt)

In [ ]:
!cat obj/video_0_1000.txt

In [ ]:
# #Split the dataset into train-val
# from sklearn.model_selection import GroupKFold
# kf = GroupKFold(n_splits = 10) 
# df = df.reset_index(drop=True)
# df['fold'] = -1
# for fold, (train_idx, val_idx) in enumerate(kf.split(df, y = df.video_id.tolist(), groups=df.sequence)):
#     df.loc[val_idx, 'fold'] = fold
# display(df.fold.value_counts())

In [ ]:
val_df = df[df["video_id"].isin([2])]
train_df = df[df["video_id"].isin([0, 1])]
print(train_df.shape)
print(val_df.shape)

In [ ]:
# Move labels from obj to test directory
def mv_labels (row):
    old_path = row.label_path
    filename = row.label_path.split('/')[-1]
    new_path = '/'.join(row.label_path.split('/')[:-2]) + '/test/' + filename
    row['label_path'] = new_path
    shutil.move(old_path, new_path)
    return row

val_df = val_df.apply(lambda x: mv_labels(x), axis=1)
val_df.head(2)

In [ ]:
# Copy images to working directory
'''
Labels and images must have the same name:
Images: obj/image_XX.jpg
Labels: obj/image_XX.txt
'''
def copy_images (row):
    old_path = row.image_path
    filename = row.label_path.split('/')[-1][:-4] + '.jpg'
    new_path = '/'.join(row.label_path.split('/')[:-1]) + '/' + filename
    shutil.copy(old_path, new_path)
val_df.apply(lambda x: copy_images(x), axis=1)
train_df.apply(lambda x: copy_images(x), axis=1)

In [ ]:
#Verify
!ls obj/*.jpg | wc -l
!ls obj/*.txt | wc -l
!ls test/*.jpg | wc -l
!ls test/*.txt | wc -l

In [ ]:
# Generate train.txt and test.txt files
%cd ../
train_images = glob.glob('data/obj/*.jpg')
f = open('./data/train.txt', 'w')
annot = [os.path.join(os.getcwd(),t) + ('\n' if i<len(train_images)-1 else '') for i, t in enumerate(train_images)]
annot = ''.join(annot)
annot = annot.strip()
f.write(annot)

val_images = glob.glob('data/test/*.jpg')
f = open('./data/test.txt', 'w')  
annot = [os.path.join(os.getcwd(),t) + ('\n' if i<len(val_images)-1 else '') for i, t in enumerate(val_images)]
annot = ''.join(annot)
annot = annot.strip()
f.write(annot)

In [ ]:
#Verify
!cat data/train.txt | wc -l
!cat data/test.txt | wc -l

In [ ]:
#Visualization
np.random.seed(1)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

df2 = train_df[(train_df.num_bbox>0)].sample(100) # takes samples with bbox

for idx in range(10):
    row = df2.iloc[idx]
    img           = load_image(row.image_path)
    image_height  = row.height
    image_width   = row.width
    f = open(row.label_path)
    bboxes_yolo = np.asarray([[float(a) for a in l[1:].strip().split(' ')] for l in f.readlines()])

    names         = ['starfish']*len(bboxes_yolo)
    labels        = [0]*len(bboxes_yolo)

    plt.figure(figsize = (12, 8))
    plt.imshow(draw_bboxes(img = img,
                           bboxes = bboxes_yolo, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = 'yolo',
                           line_thickness = 2))
    plt.axis('OFF')
    plt.show()

In [ ]:
# #Modify YOLO's configuration file as per our data
# !sed -i 's/subdivisions=8/subdivisions=64/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/max_batches = 500500/max_batches = 8000/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/steps=400000,450000/steps=6400,7200/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/classes=80/classes=1/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/filters=255/filters=18/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/objectness_smooth=1/objectness_smooth=0/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/scale_x_y=2.0/scale_x_y=1.05/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/activation=logistic/activation=linear/g' ./cfg/yolov4x-mish.cfg
# !sed -i 's/iou_thresh=0.2/iou_thresh=1.0/g' ./cfg/yolov4x-mish.cfg
# !sed 'iou_loss=ciou d' ./cfg/yolov4x-mish.cfg
# !sed 'iou_normalizer=0.05 d' ./cfg/yolov4x-mish.cfg
# !sed 'new_coords=1 d' ./cfg/yolov4x-mish.cfg

In [ ]:
# !sed -i 's/subdivisions=64/subdivisions=32/g' ./cfg/yolov4-custom.cfg 
# !sed -i 's/width=1024/width=832/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/height=1024/height=832/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/max_batches = 12000/max_batches = 5000/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/steps=9600,10800/steps=4000,4500/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/classes=80/classes=1/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/filters=255/filters=18/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/angle = 0/angle= 30/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/filters=255/filters=18/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/layers = 23/layers = 54/g' ./cfg/yolov4-custom.cfg
# !sed -i 's/stride = 4/stride= 2/g' ./cfg/yolov4-custom.cfg
# !sed 'random=1 d' ./cfg/yolov4-custom.cfg

In [ ]:
!sed -i 's/subdivisions=16/subdivisions=32/g' ./cfg/yolov4-custom.cfg 
!sed -i 's/width=608/width=608/g' ./cfg/yolov4-custom.cfg
!sed -i 's/height=608/height=608/g' ./cfg/yolov4-custom.cfg
!sed -i 's/max_batches = 500500/max_batches = 5000/g' ./cfg/yolov4-custom.cfg
!sed -i 's/steps=400000,450000/steps=4000,4500/g' ./cfg/yolov4-custom.cfg
!sed -i 's/classes=80/classes=1/g' ./cfg/yolov4-custom.cfg
!sed -i 's/filters=255/filters=18/g' ./cfg/yolov4-custom.cfg
!sed 'random=1 d' ./cfg/yolov4-custom.cfg
!sed -i 's/angle = 0/angle= 30/g' ./cfg/yolov4-custom.cfg
!sed -i 's/exposure= 1.5/exposure= 2.5/g' ./cfg/yolov4-custom.cfg



In [ ]:
# Build obj.data and obj.names files 
f = open('./data/obj.data', 'w')
f.write('classes = 1\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = backup\n')
f.close()
f = open('./data/obj.names', 'w')
f.write('starfish')
f.close()

In [ ]:
# # Download YOLOv4x-MISH pre-trained model
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4x-mish.conv.166

In [ ]:
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# # Start the training
# !./darknet detector train data/obj.data cfg/yolov4x-mish.cfg yolov4x-mish.conv.166 -dont_show -map

In [ ]:
# from distutils.dir_util import copy_tree
# #copy_tree('../input/cots-last-weight-yolo','/kaggle/working/darknet/backup')
# import shutil
# src='../input/cots-last-weight-yolo/yolov4-custom_last.weights'
# dst = '/kaggle/working/darknet/yolov4-custom_last.weights'
# try:
#     #if path already exists, remove it before copying with copytree()
#     if os.path.exists(dst):
#         shutil.rmtree(dst)
#         shutil.copyfile(src, dst)
#         print("Copy File 1")
#     elif not os.path.isdir('/kaggle/working/darknet/backup'):
#         os.makedirs('/kaggle/working/darknet/backup')
#         print("folder is created!")
#     else:
#         shutil.copyfile(src, dst)
#         print("Copy File 2")
# except OSError as e:
#         #shutil.copy(source_dir_prompt, destination_dir_prompt)
#         print("No Copy due to: ", e)

In [ ]:
#!./darknet detector test data/obj.data cfg/yolov4-custom.cfg {ROOT_DIR}/-dont_show -map
#resume training
#!./darknet detector train data/obj.data cfg/yolov4-custom.cfg  -map
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg -dont_show -map

In [ ]:
# os.listdir('../../working/darknet/results')

In [ ]:
# def imShow(path):
#   import cv2
#   import matplotlib.pyplot as plt
#   %matplotlib inline

#   image = cv2.imread(path)
#   height, width = image.shape[:2]
#   resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

#   fig = plt.gcf()
#   fig.set_size_inches(18, 10)
#   plt.axis("off")
#   plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
#   plt.show()

In [ ]:

# imShow('predictions.jpg')

In [ ]:
# cd /kaggle/working/darknet

In [ ]:
# bboxes=[335,91,27,27]

In [ ]:
# import cv2 as cv

In [ ]:
# def load_model(conf=0.25, iou=0.50):
#     net = cv.dnn.readNet(f'/kaggle/input/cots-yolov4-final/cots_yolov4_final_weights/yolov4-custom.cfg',
#                             f'/kaggle/input/yolocotsbest/yolov4-custom_best.weights')
#     net = cv.dnn_DetectionModel(net)
#     net.setInputParams(size=(416, 416), scale=1/255, swapRB=True)
#     return net

In [ ]:
# net = load_model(conf=0.25, iou=0.40)

In [ ]:
# confThreshold = 0.25
# confthre = 0.25
# IMG_SIZE=416
# with open('/kaggle/input/objdata/obj.names', 'rt') as f:
#     names = f.read().rstrip('\n').split('\n')

# def predict(net, img, size=IMG_SIZE):
#     confs = []
#     bboxes = []
#     height, width = img.shape[:2]
#     bbclasses, scores, bboxes = net.detect(img, confThreshold=confThreshold, nmsThreshold=0.4)
#     print( scores, bboxes)
#     if type(bbclasses) is not tuple:
#         for classId, confidence, box in zip(bbclasses.flatten(), scores.flatten(), bboxes):
#     #         confs = '%.2f' % confidence
#     #         label = '%s: %s' % (names[classId], label)
#             left, top, width, height = box
#             if len(box):
#                 bboxes  = yolo2coco(box,height,width).astype(int)
#                 confs = '%.2f' % confidence
#             else:
#                 bboxes = []
#                 confs = []
#                 print("COCO conversion Issue!")
#         return bboxes,confs
#     else:
#         for classId, confidence, box in zip(bbclasses, scores, bboxes):
#     #         confs = '%.2f' % confidence
#     #         label = '%s: %s' % (names[classId], label)
#             left, top, width, height = box
#             if len(box):
#                 bboxes  = voc2coco(box,height,width).astype(int)
#                 print("bboxes::", bboxes)
#                 confs = '%.2f' % confidence
#             else:
#                 bboxes = []
#                 confs = []
#                 print("COCO conversion Issue!")
#         return bboxes,confs
# #     for i in range(len(bboxes)):
# #         box = bboxes[i]
# #         cls_id = int(bbclasses[i])
# #         score = scores[i]
# #     if len(bboxes):
# #         bboxes  = yolo2coco(bboxes,height,width).astype(int)
# #         confs   = score
# #         print("confs:", confs)
# #         return bboxes, confs
# #     else:
# #         return [],[]


In [ ]:
# def voc2yolo(bboxes, image_height=720, image_width=1280):
#     """
#     voc  => [x1, y1, x2, y1]
#     yolo => [xmid, ymid, w, h] (normalized)
#     """
    
#     bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
#     bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
#     bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
#     w = bboxes[..., 2] - bboxes[..., 0]
#     h = bboxes[..., 3] - bboxes[..., 1]
    
#     bboxes[..., 0] = bboxes[..., 0] + w/2
#     bboxes[..., 1] = bboxes[..., 1] + h/2
#     bboxes[..., 2] = w
#     bboxes[..., 3] = h
    
#     return bboxes
# def yolo2voc(bboxes, image_height=720, image_width=1280):
#     """
#     yolo => [xmid, ymid, w, h] (normalized)
#     voc  => [x1, y1, x2, y1]
    
#     """ 
#     bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
#     bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
#     bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
#     bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
#     bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
#     return bboxes
# def coco2yolo(bboxes, image_height=720, image_width=1280):
#     """
#     coco => [xmin, ymin, w, h]
#     yolo => [xmid, ymid, w, h] (normalized)
#     """
    
#     bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
#     # normolizinig
#     bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
#     bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
#     # converstion (xmin, ymin) => (xmid, ymid)
#     bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
#     return bboxes
# def yolo2coco(bboxes, image_height=416, image_width=416):
#     """
#     yolo => [xmid, ymid, w, h] (normalized)
#     coco => [xmin, ymin, w, h]
    
#     """ 
#     bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
#     # denormalizing
#     bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
#     bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
#     # converstion (xmid, ymid) => (xmin, ymin) 
#     bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
#     return bboxes

# def voc2coco(bboxes, image_height=720, image_width=1280):
#     bboxes  = voc2yolo(bboxes, image_height, image_width)
#     bboxes  = yolo2coco(bboxes, image_height, image_width)
#     return bboxes

In [ ]:
# !pip install -q /kaggle/input/bbox-lib-ds

In [ ]:
# from PIL import Image
# from numpy import asarray
  
  

# img = Image.open('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/1910.jpg')
  
# # asarray() class is used to convert
# # PIL images into NumPy arrays
# numpydata = asarray(img)

In [ ]:

# bboxes, confs  = predict(net, numpydata, size=IMG_SIZE)

In [ ]:
# print(bboxes,confs)

In [ ]:
# def format_prediction(bboxes, confs):
#     annot = ''
#     if len(bboxes)>0:
#         for idx in range(len(bboxes)):
#             xmin, ymin, w, h = bboxes[idx]
#             conf             = confs
#             annot += f'{conf} {xmin} {ymin} {w} {h}'
#             annot +=' '
#         annot = annot.strip(' ')
#     return annot

In [ ]:
#  annot= format_prediction(bboxes, confs)

In [ ]:
# annot

In [ ]:
# print(bboxes,confs)

In [ ]:
# colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
#           for idx in range(1)]
# print(colors)

# #df2 = train_df[(train_df.num_bbox>0)].sample(100) # takes samples with bbox

    
# img = load_image('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/1875.jpg')
# image_height  = 720
# image_width   = 1280
    
# bboxes_yolo = [[ 35,216 ,46 , 44],[681,602 ,50 ,86]]

# names         = ['starfish']*len(bboxes_yolo)
# labels        = [0]*len(bboxes_yolo)

# plt.figure(figsize = (12, 8))
# plt.imshow(draw_bboxes(img = img,
#                            bboxes = bboxes_yolo, 
#                            classes = names,
#                            class_ids = labels,
#                            class_name = True, 
#                            colors = colors, 
#                            bbox_format = 'yolo',
#                            line_thickness = 2))
# plt.axis('OFF')
# plt.show()
    



In [ ]:
# img

In [ ]:
# k

In [ ]:
# !pip install darknetpy



In [ ]:
# det=load_model()

In [ ]:
# GPU=1 pip install darknetpy

In [ ]:
# !pip install https://github.com/sicara/tf2-yolov4/archive/master.zip

In [ ]:
# !convert-darknet-weights /kaggle/input/yolocotsbest/yolov4-custom_best.weights -o yolov4.h5

In [ ]:
#  pip install --upgrade pip

<a href="/kaggle/working/darknet/backup"> Download File </a>
